In [ ]:
#1.主题模型的sklearn实现

from sklearn.feature_extraction.text import TfidfTransformer

import jieba
from sklearn.feature_extraction.text import CountVectorizer
#先弄出文档词频矩阵，然后进行相应的模型训练
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
rawfile=[]#空格分隔开的文本
for i in range(len(df2)):
    rawfile.append(' '.join(df2['fenci'][i]))
#print(rawfile) 
#rawfile listoflist ,每一项为用空格连接的一篇文档
countvec = CountVectorizer(min_df = 5)#最小词频
X = countvec.fit_transform(rawfile)#稀疏矩阵

transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X)
#基于词频矩阵X计算TF-IDF值
#---------------------------------
#设定LDA模型
from sklearn.decomposition import LatentDirichletAllocation
n_topics=3#提取主题个数
ldamodel = LatentDirichletAllocation(n_components= n_topics)
#拟合LDA模型
ldamodel.fit(tfidf)
#拟合后的模型实质
print(ldamodel.components_.shape)#(10,705)
ldamodel.components_[:2]#前两条，每个主题中每个词条出现的概率

#打印主题词
def print_top_words(model,feature_names,n_top_words):
    for topic_idx,topic in enumerate(model.components_):
        print('Topic #%d:'% topic_idx)
        print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

n_top_words = 10
tf_feature_names = countvec.get_feature_names()
print_top_words(ldamodel, tf_feature_names, n_top_words)



In [ ]:
#2.主题模型的gensim实现


#文档预处理
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
filelist=[]
for i in range(len(df2)):
    filelist.append(df2['fenci'][i])
#生成文档对应的字典和bow稀疏矩阵
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist] # 仍为list in list  

tfidf_model = models.TfidfModel(corpus) # 建立TF-IDF模型  
corpus_tfidf = tfidf_model[corpus] # 对所需文档计算TF-IDF结果
corpus_tfidf

#拟合LDA模型
from gensim.models.ldamodel import LdaModel
# 列出所消耗的时间备查
%time ldamodel = LdaModel(corpus, id2word = dictionary, num_topics = 3, passes = 10) 
#列出最重要的若干个主题
ldamodel.print_topics(num_topics = 20,num_words = 10)

#计算各语料的LDA模型值
corpus_lda = ldamodel[corpus_tfidf] # 此处应当使用和模型训练时相同类型的矩阵
for doc in corpus_lda:
    print(doc)
ldamodel.get_topics()#list of list 每个主题中每个词所对应的一个概率矩阵

# 检索和文本内容最接近的主题
 # 检索和0.txt最接近的主题
query_bow = dictionary.doc2bow(df2['fenci'][0]) # 频数向量
query_tfidf = tfidf_model[query_bow] # TF-IDF向量
print("转换后：", query_tfidf[:10])
ldamodel.get_document_topics(query_bow) # 需要输入和文档对应的bow向量
# 检索和文本内容最接近的主题

ldamodel[query_tfidf]



In [ ]:

# 对gensim的LDA结果作呈现
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
print('--------------')
pyLDAvis.disable_notebook() # 关闭notebook支持后，可以看到背后所生成的数据



In [1]:
#用原始词频矩阵拟合LDA模型
#2.主题模型的gensim实现


#文档预处理
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
filelist=[]
for i in range(len(df2)):
    filelist.append(df2['fenci'][i])
#生成文档对应的字典和bow稀疏矩阵
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist] # 仍为list in list  


#拟合LDA模型
from gensim.models.ldamodel import LdaModel
# 列出所消耗的时间备查
%time ldamodel = LdaModel(corpus, id2word = dictionary, num_topics = 15, passes = 10) 
#列出最重要的若干个主题
ldamodel.print_topics(num_topics = 15,num_words = 10)

#计算各语料的LDA模型值
corpus_lda = ldamodel[corpus] # 此处应当使用和模型训练时相同类型的矩阵
for doc in corpus_lda:
    print(doc)
ldamodel.get_topics()#list of list 每个主题中每个词所对应的一个概率矩阵

# 检索和文本内容最接近的主题
 # 检索和0.txt最接近的主题
query_bow = dictionary.doc2bow(df2['fenci'][7]) # 频数向量
ldamodel.get_document_topics(query_bow) # 需要输入和文档对应的bow向量
# 检索和文本内容最接近的主题

ldamodel[query_bow]
ldamodel.save('C:/EdmsData/ldamodel.txt')


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache


['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9']


Loading model cost 1.362 seconds.
Prefix dict has been built succesfully.
C:\Tools\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 2.83 s
[(2, 0.99918199)]
[(2, 0.99918199)]
[(1, 0.1047532), (11, 0.89464784)]
[(6, 0.99924606)]
[(1, 0.99903977)]
[(8, 0.99853247)]
[(3, 0.058429006), (10, 0.94022316)]
[(11, 0.99900174)]
[(3, 0.9983539)]
[(11, 0.99971533)]
[(3, 0.081012875), (6, 0.91799438)]
[(14, 0.99831831)]
[(0, 0.99973357)]
[(8, 0.027190065), (13, 0.97153544)]
[(8, 0.99817348)]
[(13, 0.98962379)]
[(13, 0.99911946)]
[(7, 0.99809909)]
[(8, 0.99834216)]
[(0, 0.64452708), (3, 0.05509999), (6, 0.24650821), (11, 0.053195063)]
[(2, 0.044811487), (3, 0.9541555)]
[(10, 0.99783802)]
[(1, 0.99942774)]
[(6, 0.99861932)]
[(3, 0.073396884), (11, 0.92589217)]
[(10, 0.99948663)]
[(3, 0.99930501)]
[(13, 0.9996354)]
[(9, 0.99971247)]
[(3, 0.99934363)]
[(6, 0.99966997)]


In [ ]:
#1.主题模型的sklearn实现

from sklearn.feature_extraction.text import TfidfTransformer

import jieba
from sklearn.feature_extraction.text import CountVectorizer
#先弄出文档词频矩阵，然后进行相应的模型训练
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
rawfile=[]#空格分隔开的文本
for i in range(len(df2)):
    rawfile.append(' '.join(df2['fenci'][i]))
#print(rawfile) 
#rawfile listoflist ,每一项为用空格连接的一篇文档
countvec = CountVectorizer(min_df = 0.1)#最小词频
X = countvec.fit_transform(rawfile)#稀疏矩阵

#基于词频矩阵X计算TF-IDF值
#---------------------------------
#设定LDA模型
from sklearn.decomposition import LatentDirichletAllocation
n_topics=3#提取主题个数
ldamodel = LatentDirichletAllocation(n_components= n_topics)
#拟合LDA模型
docres = ldamodel.fit_transform(X)
#拟合后的模型实质
print(docres)
print(ldamodel.components_.shape)#(10,705)
print(ldamodel.components_)#前两条，每个主题中每个词条出现的概率



#打印主题词

def print_top_words(model,feature_names,n_top_words):
    for topic_idx,topic in enumerate(model.components_):
        print('Topic #%d:'% topic_idx)
        print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

n_top_words = 10
tf_feature_names = countvec.get_feature_names()
print_top_words(ldamodel, tf_feature_names, n_top_words)





In [23]:
#1.主题模型的sklearn实现

from sklearn.feature_extraction.text import TfidfTransformer

import jieba
from sklearn.feature_extraction.text import CountVectorizer
#先弄出文档词频矩阵，然后进行相应的模型训练
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
rawfile=[]#空格分隔开的文本
for i in range(len(df2)):
    rawfile.append(' '.join(df2['fenci'][i]))
#print(rawfile) 
#rawfile listoflist ,每一项为用空格连接的一篇文档
countvec = CountVectorizer(min_df = 0.1)#最小词频
X = countvec.fit_transform(rawfile)#稀疏矩阵

#基于词频矩阵X计算TF-IDF值
#---------------------------------
#设定LDA模型
from sklearn.decomposition import LatentDirichletAllocation
n_topics = range(5, 75, 5)
perplexityLst = [1.0]*len(n_topics)
 
#训练LDA并打印训练时间
lda_models = []
for idx, n_topic in enumerate(n_topics):
    lda = LatentDirichletAllocation(n_topics=n_topic,max_iter=20,learning_method='batch',evaluate_every=200,verbose=0)
  
    lda.fit(X)
    perplexityLst[idx] = lda.perplexity(X)
    lda_models.append(lda)
    print("# of Topic: %d, " % n_topics[idx])

    print("Perplexity Score %0.3f" % perplexityLst[idx])
#打印最佳模型
best_index = perplexityLst.index(min(perplexityLst))
best_n_topic = n_topics[best_index]
best_model = lda_models[best_index]
print("Best # of Topic: ", best_n_topic)

#设定LDA模型
from sklearn.decomposition import LatentDirichletAllocation
n_topics=best_n_topic#提取主题个数
ldamodel = LatentDirichletAllocation(n_components= n_topics)
#拟合LDA模型
docres = ldamodel.fit_transform(X)
#拟合后的模型实质
print(docres)
print(ldamodel.components_.shape)#(10,705)
print(ldamodel.components_)#前两条，每个主题中每个词条出现的概率



#打印主题词

def print_top_words(model,feature_names,n_top_words):
    for topic_idx,topic in enumerate(model.components_):
        print('Topic #%d:'% topic_idx)
        print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

n_top_words = 10
tf_feature_names = countvec.get_feature_names()
print_top_words(ldamodel, tf_feature_names, n_top_words)




['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9']


C:\Tools\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


# of Topic: 5, 
Perplexity Score 407.072


C:\Tools\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


# of Topic: 10, 
Perplexity Score 400.574


...

# of Topic: 65, 
Perplexity Score 425.078


C:\Tools\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


# of Topic: 70, 
Perplexity Score 428.017
Best # of Topic:  15


C:\Tools\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


[[  9.96512451e-05   9.96513868e-05   9.96513972e-05   9.96513158e-05
    9.96512431e-05   9.96512332e-05   9.96512396e-05   9.98604882e-01
    9.96513181e-05   9.96513638e-05   9.96513252e-05   9.96513749e-05
    9.96512387e-05   9.96512409e-05   9.96512445e-05]
 [  9.96512451e-05   9.96513868e-05   9.96513972e-05   9.96513158e-05
    9.96512431e-05   9.96512332e-05   9.96512396e-05   9.98604882e-01
    9.96513181e-05   9.96513638e-05   9.96513252e-05   9.96513749e-05
    9.96512387e-05   9.96512409e-05   9.96512445e-05]
 [  6.69344237e-05   6.69345399e-05   6.69345723e-05   6.69345293e-05
    6.69344289e-05   6.69344139e-05   6.69344200e-05   6.69348245e-05
    6.69345546e-05   9.99062917e-01   6.69345096e-05   6.69345335e-05
    6.69344195e-05   6.69344262e-05   6.69344217e-05]
 [  7.99360728e-05   7.99362877e-05   7.99362331e-05   7.99361756e-05
    7.99360816e-05   7.99360651e-05   7.99360716e-05   9.59607592e-01
    7.99361950e-05   3.93532383e-02   7.99362226e-05   7.99362136e-0